In [1]:
import numpy as np
import rasterio
import os
from pathlib import Path

This notebook calculates neighborhood averages of the prepared datasets.

Pixels in a circle 2,5 and 10 kilometer distance around each pixel are put into zone 2,3 and 4. Zone 1 is the pixel itself. 
The zones are exclusive, so zone 3 is only the pixels between 2 and 5 km distance. 

In [2]:
# import paths. 
from rural_beauty.config import CLC_coverage_EU_dir, DEM_EU_range, windpower_EU_raster, streets_EU_raster

This is the rural_beauty module


In [3]:
# import functions create_circular_disc and get_zone_mean
from rural_beauty import blur_circular_disc
blur_circular_disc.create_circular_disc(5, 6)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [4]:
layers = [item for item in  os.listdir(CLC_coverage_EU_dir) if item.endswith('.tif')]
print(f"There are {len(layers)} layers in {CLC_coverage_EU_dir}")
print(layers)

There are 47 layers in /h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/clc/layer_coverage_EU
['code_offen.tif', 'code_abbau.tif', 'code_salzw.tif', 'code_landwi.tif', 'code_hafen.tif', 'code_bebau.tif', 'code_kraut.tif', 'code_feucht.tif', 'code_spfr.tif', 'code_stoer.tif', 'code_natur.tif', 'code_strbah.tif', 'code_stgrue.tif', 'code_weide.tif', 'code_stadt.tif', 'code_sand.tif', 'code_nwald.tif', 'code_acker.tif', 'code_suempf.tif', 'code_gewage.tif', 'code_selten.tif', 'code_wanat.tif', 'code_gezei.tif', 'code_flug.tif', 'code_fels.tif', 'code_obswei.tif', 'code_gwlf.tif', 'code_lwald.tif', 'code_siedl.tif', 'code_seemee.tif', 'code_schatt.tif', 'code_natgru.tif', 'code_seen.tif', 'code_noveg.tif', 'code_wein.tif', 'code_wald.tif', 'code_geholz.tif', 'code_strauc.tif', 'code_moor.tif', 'code_dorf.tif', 'code_indgew.tif', 'code_gewae.tif', 'code_obst.tif', 'code_heide.tif', 'code_mwald.tif', 'code_meer.tif', 'code_semage.tif']


In [5]:
zone_dict = {
    DEM_EU_range                           : ["zone2", "zone3","zone1_2", "zone3_4"], 
    CLC_coverage_EU_dir / "code_obst.tif"  : ["zone1_4"], 
    CLC_coverage_EU_dir / "code_wald.tif"  : ["zone1_4"],
    CLC_coverage_EU_dir / "code_natgru.tif": ["zone2", "zone1_2"], 
    CLC_coverage_EU_dir / "code_acker.tif" : ["zone1_4", "zone1_2"], 
    CLC_coverage_EU_dir / "code_stoer.tif" : ["zone1_2", "zone2", "zone3"], 
    CLC_coverage_EU_dir / "code_noveg.tif" : ["zone2"], 
    windpower_EU_raster                    : ["zone1_4"], 
    streets_EU_raster                      : ["zone1_2"], 

}

In [7]:
def modify_filename(file_path:str, add_string:str, new_folder:str) -> str:
    # Split the filename and the extension
    base, extension = os.path.splitext(file_path)

    # Get directory and basename
    dir_name, base_name = os.path.split(base)

    # Add the string to the basename
    new_base_name = f"{base_name}_{add_string}"

    # Construct the new path
    new_full_path = os.path.join(dir_name, new_folder, new_base_name + extension)

    # Ensure the directory exists
    new_directory = os.path.dirname(new_full_path)
    os.makedirs(new_directory, exist_ok=True)

    return new_full_path

In [9]:
# Create the circular disc
def write_specific_zone_mean(input_dict:str) -> None:
    overall_radius = 10
    zone_kernels = {
    "zone4" : blur_circular_disc.create_circular_disc(9.9, overall_radius) - blur_circular_disc.create_circular_disc(4.9, overall_radius),
    "zone3" : blur_circular_disc.create_circular_disc(4.9, 5) - blur_circular_disc.create_circular_disc(1.9, 5),
    "zone2" : blur_circular_disc.create_circular_disc(1.9, 2) - blur_circular_disc.create_circular_disc(0.9, 2),
    "zone1_4" : blur_circular_disc.create_circular_disc(9.9, overall_radius),
    "zone1_2" : blur_circular_disc.create_circular_disc(1.9, 2),
    "zone3_4" : blur_circular_disc.create_circular_disc(9.9, overall_radius) - blur_circular_disc.create_circular_disc(1.9, overall_radius)
    }
    
    for input_path, zone_names in input_dict.items():
        
        with rasterio.open(input_path) as raster_in:
            for zone_name in zone_names:
                # Define the output path within the new directory
                output_path = modify_filename(input_path, zone_name, 'neighborhood')

                if zone_name == 'zone1':
                    print(f"Zone 1 is the same as input data. Skipped :{output_path}")
                    continue

                kernel = zone_kernels[zone_name]
                meta = raster_in.meta.copy()
                na_value = raster_in.nodata
                array_in = raster_in.read(1)  # Reading the first band
    


                if os.path.exists(output_path):
                    print(f"{output_path} already exists skipping...")
                    continue
    
                print(output_path)
                output_array = blur_circular_disc.get_zone_mean(array_in, na_value, kernel)
        
                # Create a new raster file for output
                meta.update(dtype=rasterio.float32, count=1)  # Update meta if necessary
        
                with rasterio.open(output_path, 'w', **meta) as raster_out:
                    raster_out.write(output_array, 1)  # Write output array to the first band

In [ ]:
write_specific_zone_mean(zone_dict)

/h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/dem/neighborhood/DEM_EU_range_zone2.tif already exists skipping...
/h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/dem/neighborhood/DEM_EU_range_zone3.tif already exists skipping...
/h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/dem/neighborhood/DEM_EU_range_zone1_2.tif already exists skipping...
/h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/dem/neighborhood/DEM_EU_range_zone3_4.tif already exists skipping...
/h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/clc/layer_coverage_EU/neighborhood/code_obst_zone1_4.tif already exists skipping...
/h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/clc/layer_coverage_EU/neighborhood/code_obst_zone2.tif
Zone 1 is the same as input data. Skipped :/h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/clc/layer_coverage_EU/neighborhood/code_obst_zone1.tif
/h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/clc/layer_coverage_EU/neighborhood/code_obst_zone3.